Script de select datos MySQL en cluster multidomain

In [50]:
!pip install mysql-connector==2.1.7
!pip install pandas
!pip install sqlalchemy
#requiere instalación adicional, consultar https://github.com/PyMySQL/mysqlclient
!pip install mysqlclient
!pip install numpy
!pip install pymysql

In [51]:
import mysql.connector
from mysql.connector import Error
import pandas as pd
import numpy as np
import sqlalchemy
import time
import matplotlib.pyplot as plt
import psutil
import random

from mysql.connector.cursor import MySQLCursor

In [52]:
#Los resultados de medidas de tiempo en carga por dominios se almacenan en estos objetos.
#Se itera durante 100 iteraciones para sacar medias
#repeticiones
repeats = 100

In [53]:
#Ficheros de salida
resultados_etl_select = '../Results/MySQL/MySQLSelect_test_{}.csv'

In [54]:
def save_results_to_csv(results,file):
    #Guardamos los resultados en csv
    from datetime import datetime
    
    csv_df = pd.DataFrame(results, columns=['Registros', 'Tiempo', 'CPU','Memoria'])
    dia = datetime.now().strftime("%d%m%Y_%H_%M_%S")
    csv_df.to_csv(file.format(str(dia)))

In [55]:
conn = mysql.connector.connect(host='localhost',
                                    port=6446,
                                    database='CustomerProfileDomainSchema',
                                    user='root',                                       
                                    password='mysql')
if conn.is_connected():
        conn.autocommit = False #Apenas se observa diferencia de rendimiento
        print('Connected to MySQL database')
    
cursor = conn.cursor()  
    
def connection_open():
   
    #InnoDB Cluster
    conn = mysql.connector.connect(host='localhost',
                                    port=6446,
                                    database='CustomerProfileDomainSchema',
                                    user='root',                                       
                                    password='mysql')


    if conn.is_connected():
        conn.autocommit = False #Apenas se observa diferencia de rendimiento
        print('Connected to MySQL database')
    
    cursor = conn.cursor()  

Connected to MySQL database


In [56]:
cursor = conn.cursor(buffered=True)
cursor.execute('set global group_replication_transaction_size_limit = 300000000;')
cursor.close()

True

# Select test multidomain

In [57]:
#Obtenemos los posibles valores de pais. Se iterará por ellos para cambiar en bucle los registros
cursor = conn.cursor(buffered=True)
cursor.execute("""SELECT DISTINCT (PartyId) FROM CustomerProfileDomainSchema.CustomerProfile""")
partyId_list= []
for partyId in cursor.fetchall():
    partyId_list.append(partyId[0])
print(len(partyId_list))
cursor.close()

1000000


True

In [58]:
time_inicial = 0
time_final = 0
registers = []
 
for iteracion in range(0,repeats): 
    select_query = """SELECT * FROM CustomerProfileDomainSchema.CustomerProfile cp
INNER JOIN CurrentAccountDomainSchema.CurrentAccount ca ON ca.PartyId = cp.PartyId
INNER JOIN CurrentAccountDomainSchema.AccountInfo ai ON ai.AccountId = ca.AccountId
INNER JOIN PositionKeepingDomainSchema.PositionKeeping pk ON pk.AccountId = ca.AccountId
INNER JOIN PositionKeepingDomainSchema.Amount am ON am.AmountId = pk.AmountId
INNER JOIN PositionKeepingDomainSchema.CreditLine cl ON cl.CreditLineId = pk.CreditLineId
INNER JOIN PositionKeepingDomainSchema.Currency cr ON cr.CurrencyId = am.CurrencyId
WHERE cp.PartyId = {}""".format(random.choice(partyId_list))
    cursor = conn.cursor(buffered=True)
    time_inicial = time.time()
    cursor.execute(select_query)
    time_final = time.time() 
    used_cpu = psutil.cpu_percent()
    mem_used = psutil.virtual_memory().percent
    # Tupla con numero de registros, tiempo parcial de la transacción y tiempo acumulado de trxs
    total_time = round(time_final - time_inicial,3)
    registers.append((iteracion + 1, total_time ,used_cpu, mem_used))
    #print((iteracion + 1 , total_time ,used_cpu, mem_used))
    cursor.close()
conn.close()

In [59]:
#Guardamos los resultados Customer Profile
save_results_to_csv(registers,resultados_etl_select)

In [60]:
conn.close()
print('Conexion cerrada')

Conexion cerrada
